In [1]:
######################################################
# Sample elements to find left&right edges on the road
# Uses Hough transformation to find edge lines 
# Test video with road excluded
######################################################

In [7]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [8]:
SOURCE_PATH = './test_images/test.mp4'
OUTPUT_FOLDER = './output/'

In [9]:
video_capture = cv2.VideoCapture(SOURCE_PATH)

WIDTH = int(video_capture.get(3))
HEIGHT = int(video_capture.get(4))

In [10]:
def make_coordinates(img, line_parameters):
    
    """Calculate line coordinates from average line parameters"""
    
    slope, intercept = line_parameters
    y1 = img.shape[0]
    y2 = int(y1*(3/5))
    # y = mx + b; x = (y-b) / m
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return np.array([x1, y1, x2, y2])


def get_edges(img, low=50, high=150, kernel=(5, 5)):
    
    """Process canny image"""
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    blurred = cv2.GaussianBlur(gray, kernel, 1)
    edged = cv2.Canny(blurred, low, high)
    
    return edged


def get_roi(img, height=HEIGHT):
    
    """Put mask with region of interest on the image """
    
    polygons = np.array([
        [(200, height), (1100, height), (550,250)]
    ])
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, polygons, 255)
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img


def show_lines(img, lines):
    line_image = np.zeros_like(img)
    if lines is not None:
        for x1, y1, x2, y2 in lines:
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 4)
    return line_image


def avg_slope_intercept(img, lines):
    
    """Calculate average line parameters from results of Hough transform"""
    
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4) # unpack the elements
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        # print(parameters)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
        # print(f"left {left_fit}")
        # print(f"right {right_fit}")
    left_fit_avg = np.average(left_fit, axis=0)
    right_fit_avg = np.average(right_fit, axis=0)
    left_line = make_coordinates(img, left_fit_avg)
    right_line = make_coordinates(img, right_fit_avg)
    return np.array([left_line, right_line])


In [11]:
while video_capture.isOpened():
    success, frame = video_capture.read()
    
    canny = get_edges(frame, 50, 150)
    masked = get_roi(canny)
    # lines: 2d array in form [[x1, y1, x2, y2]]
    lines = cv2.HoughLinesP(masked, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)
    avg_lines = avg_slope_intercept(frame, lines)
    line_image = show_lines(frame, avg_lines)
    comb_img = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
    
    cv2.imshow('Video', comb_img)
    
    k = cv2.waitKey(1) & 0xFF
    
    if k == ord('q'):
        break
    
    elif k == ord('s'):
        fp = OUTPUT_FOLDER + 'image.jpg'
        cv2.imwrite(fp, comb_img)

          
video_capture.release()
cv2.destroyAllWindows()
    
    